#### 라이브러리 import

In [ ]:
import ultralytics
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco
import pandas as pd
import os

#### COCO형식을 YOLO형식으로 변환
- (x_min, y_min, w, h) -> (x_center, y_centre, w, h)
- 각 이미지마다 annotation정보가 {save_dir}/labels라는 폴더에 txt파일로 저장됩니다.

In [ ]:
convert_coco(
    labels_dir = '/data/ephemeral/datasets/annotations',
    save_dir = '/data/ephemeral/coco2yolo',
    use_keypoints= False,
    use_segments= False,
    cls91to80 = False,
    lvis = False
)

#### 사용할 모델 선정

In [ ]:
model_name = 'yolo11x'

#### 모델 및 pretrain된 가중치 불러오기

In [ ]:
model = YOLO(f"{model_name}.yaml").load(f"{model_name}.pt")

#### 모델 빌드 및 하이퍼 파라미터 설정
- 배치사이즈, Epoch, 디바이스, 옵티마이저 및 여러 하이퍼 파라미터 등을 지정할 수 있습니다.
- 추가적으로 밝기, 기울이기, Mosaic, Flip, Mix up 등 다양한 증강의 사용여부와 확률을 지정할 수 있습니다.

In [ ]:
train_results = model.train(
    data = "/data/ephemeral/home/coco2yolo/rc_trash.yaml",
    batch=2,
    epochs=50,  
    imgsz=1280,  
    device=0,
    optimizer='SGD',
    lr0=0.001,
    lrf=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=5,
    project='yolo',
    name=f'{model_name}'
)

#### 학습 후 가장 성능이 좋았던 pt파일 불러오기

In [ ]:
pred_model = YOLO("/data/ephemeral/home/yolo/yolo11x/weights/best.pt")

#### Test 데이터 예측
- confidence score, IoU, TTA 사용 여부, amp 사용 여부를 predict 메서드의 인자로 지정할 수 있습니다.
- 결과를 어떤 파일로 저장할지와 어떤 결과들을 저장할지도 지정 할 수 있습니다.
- Predict의 결과들은 각 이미지별로 txt파일로 runs/detect 폴더 안에 저장됩니다.

In [ ]:
test_results = pred_model.predict(
    source="/data/ephemeral/home/coco2yolo/images/test",  
    imgsz=1280,
    augment = True,
    save = True,
    save_txt = True,
    save_conf = True,
    half = True
)

#### Submission 형태로 변환
- YOLO의 prediction 결과가 (x_center, y_center, w, h)로 나오기에 Pascal Voc 형식으로 변경해주어야 합니다.

In [ ]:
submission = pd.DataFrame()

prediction_strings = []
file_names = []

txt_dir = '/data/ephemeral/home/runs/detect/predict/labels'
txt_files = sorted([f for f in os.listdir(txt_dir) if f.endswith('.txt')])

img_width, img_height = 1024, 1024

for txt_file in txt_files:
    txt_path = os.path.join(txt_dir, txt_file)
    prediction_by_image = []
    
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            a = line.strip().split()
            
            cls = int(a[0])
            conf = float(a[-1])
            
            center_x = float(a[1]) * img_width
            center_y = float(a[2]) * img_height
            width = float(a[3]) * img_width
            height = float(a[4]) * img_height
            
            xmin = float(center_x - width / 2)
            ymin = float(center_y - height / 2)
            xmax = float(center_x + width / 2)
            ymax = float(center_y + height / 2)
            
            prediction_by_image.append(f"{cls} {conf} {xmin} {ymin} {xmax} {ymax}")

    prediction_string = ' '.join(prediction_by_image)
    file_name = f"test/{txt_file.replace('.txt', '.jpg')}"
    
    prediction_strings.append(prediction_string)
    file_names.append(file_name)

submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

submission.to_csv(os.path.join('/data/ephemeral/home/submission', 'yolo11x.csv'), index=None)

submission.head(3)
